<img src="https://github.com/bjorneju/IIT_wiki_tutorial/blob/main/notebooks/figures/Intrinsicality.png?raw=true" width=500/>

# Intrinsicality
This second tutorial notebook delves deeper into what it means to "apply intrinsicality" in IIT. Following the first tutorial notebook, as well as introductions in the [postulate page]{link} and the [unfolding page]{link} of the IIT wiki, we will explore the following concepts:

1. Causal conditioning (or pinning) the background conditions,
2. Isolating a candidate substrate of consciousness,
3. Computing selectivity,
5. Adjusting causal power/informativeness of a substrate to make it intrinsic causal power for a candidate substrate of consciousness.

## Import packages
To get going, we need to import packages needed for our basic examples

In [9]:
import pyphi
from tutorial_functions import visualization as viz
from tutorial_functions import utils
import numpy as np

## Initialize the substrate
First, we recreate the substrate that was introduced in Existence notebook (and Fig. 1 of the IIT 4.0 article). As a reminder, the substrate is here assumed to be constituted by 3 units (labeled A, B, and C), interacting through excitatory and inhibitory connections.

In [3]:
# give names to the units
node_labels = ["A","B","C"]

# set the strength of connectivity between units
connectivity = np.array(
    [
        [-.2,0.7,0.2],
        [0.7,-.2,0.0],
        [0.0,-.8,0.2]        
    ]
)

# set the level of determinism for the units' activation function 
determinism = 4

# build the network
substrate = pyphi.network_generator.build_network(
    [pyphi.network_generator.ising.probability]*len(node_labels),
    connectivity,
    temperature=1/determinism,
    node_labels=node_labels
)

## Define the candidate complex
Next, we must specify which units constitute our candidate complex. As may become clear at a later stage, this also requires that we specify the current state of every unit in the substrate.

In [5]:
current_state = (0, 1, 1)  # A off, B on, C on
complex_units = (0, 1)  # units A (index 0) and B (index 1) constitute our candidate complex

candidate_complex = pyphi.Subsystem(substrate, current_state, complex_units)

## The question: Does a candidate complex satisfy the 1st postulate (intrinsicality)? 
1st postulate of IIT: The substrate of consciousness must have intrinsic cause–effect power:
it must take and make a difference within itself.

To answer this question we must check whether a particular candidate complex (a subset of units within the substrate) has both cause and effect power within itself. 

In [ ]:
# we can assess wether it satisfies intrinsicality
utils.assess_intrinsicality(substrate, candidate_complex)

## Isolating a candidate substrate of consciousness: conditioning on background conditions
Since we are considering the subsystem AB as our candidate complex, we are not interested in causal powers involving unit C when accounting for its potential experience. 
That is, we are looking to isolate the *intrinsic powers* of AB, as that is what IIT postulates is needed to account for its experience (if it has any at all).
To do this, we need to *causally condition* on any unit outside of the candidtae complex (in this case, unit C)

Together with causal marginalization, causal conditioning, is one of two ways of removing the causal influence of a (set of) unit(s) in a substrate. 
When we causally condition on a unit, we hold its state constant effectively removing its capacity to change. 
This makes it so that it no longer has any counterfactual states available to it.
Thus, it can no longer be a difference that can make or take a difference---it is no longer "a difference" at all.

Interestingly, causally conditioning on a unit in a substrate could alter the powers of any unit that interacts with it. 
This is because the powers of a set of physical units depends on the probabilities of conterfactual states in all units it interacts with. 
Therefore, by removing the counterfactual states of a unit, we remove the capacity of any other unit to take a difference from it and make a difference to it. 
Thus, by conditioning on all units outside a candidate substrate of consciousness, we effectively restrict its remaining causal powers to be over itself. 
That is, any remaining differences made or taken by the candidate substrate of consciousness are made or taken by units within the substrate itself.
This is a first requirement for making the power *intrinsic*.

In our example substrate, we see that unit C is currently in state 'ON'.
As we saw in the existence step, the causal powers of a substrate are fully determined by its TPM.
However, the full substrate TPM includes within it the counterfactual states of C.
To find the 
So, we first need to find the TPM of the candidate substrate, AB, given that unit C is in its state.

Let's see how conditioning on a unit outside the candidate complex affects the TPM we use to compute causal powers. 

In [8]:
# remember the full (state-by-state) TPM of the substrate ABC
viz.substrate_state_by_state(substrate)

,,A,0,1,0,1,0,1,0,1
,,B,0,0,1,1,0,0,1,1
,,C,0,0,0,0,1,1,1,1
A,B,C,,,,,,,,
0,0,0,0.17,0.02,0.56,0.08,0.03,0.00,0.11,0.02
1,0,0,0.00,0.00,0.49,0.01,0.00,0.00,0.49,0.01
0,1,0,0.01,0.48,0.01,0.33,0.00,0.10,0.00,0.07
1,1,0,0.00,0.00,0.06,0.44,0.00,0.00,0.06,0.44
0,0,1,0.44,0.06,0.00,0.00,0.44,0.06,0.00,0.00
1,0,1,0.07,0.00,0.10,0.00,0.33,0.01,0.48,0.01
0,1,1,0.01,0.49,0.00,0.00,0.01,0.49,0.00,0.00


In [33]:
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm as cm

def highlight_unit_in_state(row, unit_index, unit_state):
    # get the row index (state of the system)
    row_index = row.name
    # check if specified unit is in the specified state
    unit_in_state = row_index[unit_index] == unit_state

    def color(val, colormap):
        # compute color using colormap (blue or gray)
        rgba_color = colormap(val)
        # convert rgba color to hex
        hex_color = matplotlib.colors.rgb2hex(rgba_color)
        return hex_color

    return [
        # if this cell's column index (future state) AND row index (current state) have the specified unit in the specified state, use a gradient of blue based on cell value, else use a gradient of gray
        f'background-color: {color(value, cm.Blues)}; color: black' if (unit_in_state and idx[unit_index] == unit_state) else f'background-color: {color(value, cm.Greys)}; color: #d0d0d0'
        for idx, value in row.iteritems()
    ]


In [56]:

TPM = utils.state_by_state_tpm(substrate).round(2)

TPM.style.apply(highlight_unit_in_state, axis=1, unit_index=2, unit_state=1)


/tmp/ipykernel_584938/4052299111.py:21: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, value in row.iteritems()


,,A,0,1,0,1,0,1,0,1
,,B,0,0,1,1,0,0,1,1
,,C,0,0,0,0,1,1,1,1
A,B,C,,,,,,,,
0,0,0,0.170000,0.020000,0.560000,0.080000,0.030000,0.000000,0.110000,0.020000
1,0,0,0.000000,0.000000,0.490000,0.010000,0.000000,0.000000,0.490000,0.010000
0,1,0,0.010000,0.480000,0.010000,0.330000,0.000000,0.100000,0.000000,0.070000
1,1,0,0.000000,0.000000,0.060000,0.440000,0.000000,0.000000,0.060000,0.440000
0,0,1,0.440000,0.060000,0.000000,0.000000,0.440000,0.060000,0.000000,0.000000
1,0,1,0.070000,0.000000,0.100000,0.000000,0.330000,0.010000,0.480000,0.010000
0,1,1,0.010000,0.490000,0.000000,0.000000,0.010000,0.490000,0.000000,0.000000


In [57]:
# To condition on the background conditions (here, C ON), we first pick out the
# elements of the TPM where the background conditions are satisfied.

### FIGURE OUT HOW TO HIGHLIGHT ALL CELLS WHERE THE BACKGROUND CONDITIONS ARE SATISFIED

In [58]:
# This forms the basis for the TPM of the candidate complex (AB). That is, it
# provides us with a complete description of what state the units A and B are
# likely to end up in given that they 

In [100]:
def extract_candidate_tpm(df, background_conditions):
    # Initialize masks as arrays of True
    row_mask = np.full(df.shape[0], True)
    col_mask = np.full(df.shape[1], True)

    # Update the masks for each level specified in background_states
    for background_units, background_state in background_conditions.items():
        row_mask &= df.index.get_level_values(background_units) == background_state
        col_mask &= df.columns.get_level_values(background_units) == background_state

    # Use the masks to extract the desired subset of the data
    candidate_tpm = df.loc[row_mask, col_mask]

    # Normalize each row so it sums to 1, replace NaNs (caused by 0/0) with 0
    candidate_tpm = candidate_tpm.div(candidate_tpm.sum(axis=1), axis=0).fillna(0)

    # dropping background condition units from TPM
    candidate_tpm = candidate_tpm.droplevel(list(background_conditions.keys()))
    candidate_tpm = candidate_tpm.droplevel(list(background_conditions.keys()), axis=1)
    
    return candidate_tpm


In [102]:
background_conditions = {2: 1}
candidate_tpm = extract_candidate_tpm(TPM, background_conditions)

candidate_tpm

,A,0,1,0,1
,B,0,0,1,1
A,B,,,,
0,0,0.880000,0.120000,0.000000,0.000000
1,0,0.397590,0.012048,0.578313,0.012048
0,1,0.020000,0.980000,0.000000,0.000000
1,1,0.096386,0.674699,0.024096,0.204819


In [97]:
candidate_tpm.droplevel(list(background_conditions.keys()))

A,0,1
B,0,0
C,1,1
A,,
0,0.880000,0.120000
1,0.970588,0.029412


In [ ]:
# visualize the subsystem TPM in its state-by-state form
viz.substrate_state_by_state(substrate)

## Computing the intrinsic power as "power over oneself" 
Now that we have a TPM where the influence of (and over) units outside the candidate complex, we can compute its raw power over itself. Thus, the resulting raw power is intrinsic in the sense that it is over itself.

In [ ]:
# compute raw power of from the TPM of the candidate complex


## Computing Selectivty
While we have already computed raw causal power that is intrinsic in the sense that it is power over the same set of units that constitute the candidate complex, the power must also be *from the point of view* of the candidate complex *in the present* to fully be intrinsic. Of course, any measure we apply will necessarily be extrinsic in the sense that it is computed from our point of view as entities extrinsic to the candidate complex. However, the target is still to quantify the powers that the candidate complex has from its own point of view. To achieve this, IIT 4.0 proposes modulating the raw power "over itself" by a selectivity term.

The selectivity can be thought of as a measure of how concentrated the probability (and, thus, the raw power) of the future or past state is, given that the candidate complex is in its present state, compared to how concentrated the probability *could* be. In other words, the selectivity factor is a normalized conditional probability. However, since the maximal "concentration" of probability over a state would be achieved if the future or past state was certain (probability 1), the selectivity effectively reduces to being the conditional probability of the future or past state given the present state. 

Multiplication by the selectivity term achieves two things: First, it reduces the raw power by the probability that the intended effect (or apparent cause) would actually turn out to be correct. Second, it breaks the symmetry inherrent in the raw power (that the power of the input over the output is identical to the output over the input). Take, for example, my power to "score a freethrow" in the final seconds of my basketball match. Lets, for the sake of goodwill, assume that I make freethrows in those situations with a probability P(make freethrow | last seconds of match) = 0.6. Then, the selectivity of my potential effect ("make freethrow") would also be 0.6. This is because the maximal probability I *could* have had to for making the freethrow is P=1, and dividing by 1 makes no difference. 

Note that the selecetivity 

In [5]:
# compute selectivity for a few examples (include both cause and effect side)

## Obtaining the intrinsic power specified by a candidate substrate
Now that we have computed both the raw power specified by the candidate complex over itself, and we have seen how to quantify the selectivity of that power, we can marry the two 

In [6]:
# compute the intrinsic effect power for our candidate

In [ ]:
# compute the intrinsic cause power for our candidate